In [233]:
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [234]:

import matplotlib.pyplot as plt
import seaborn as sns

path = "./datasets/_data.csv"
rent = pd.read_csv(path)
rent.columns

Index(['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро',
       'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны',
       'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел',
       'Можно с детьми/животными', 'Дополнительно', 'Название ЖК',
       'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод',
       'Ссылка на объявление'],
      dtype='object')

In [235]:
# 'Unnamed: 0' – drop that column
rent = rent.drop(labels ='Unnamed: 0', axis= 1)

In [236]:
# 'Ссылка на объявление'
rent.drop(['Ссылка на объявление'], axis=1, inplace=True)

In [237]:
# 'Тип'
rent.drop(['Тип'], axis=1, inplace=True)

In [238]:
# 'Телефоны'
rent.drop(['Телефоны'], axis=1, inplace=True)

In [239]:
# After dropping several columns
rent.columns

Index(['ID  объявления', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'Парковка', 'Цена', 'Описание', 'Ремонт', 'Площадь комнат, м2',
       'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными',
       'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м',
       'Лифт', 'Мусоропровод'],
      dtype='object')

In [240]:
# 'Адрес' – analys
# Define 'City' column 
cities = rent['Адрес'].apply(lambda x: x.split(',')[0])
cities.unique()

array(['Москва', 'Санкт-Петербург', 'Краснодарский край'], dtype=object)

In [241]:
# 'Адрес' – remove rows with 'Москва', 'Санкт-Петербург'
rent.drop(rent[rent['Адрес'].str.startswith('Санкт-Петербург')].index, inplace = True)
rent.drop(rent[rent['Адрес'].str.startswith('Краснодарский край')].index, inplace = True)
cities = rent['Адрес'].apply(lambda x: x.split(',')[0])
cities.unique()

array(['Москва'], dtype=object)

In [242]:
# 'Дом'
def get_floor(row) :
    result = {}
    l = row['Дом'].split(',')
    if len(l) == 0 : return result
    floors = l[0].split('/')
    result['floor_flat'] = floors[0]
    result['floors_total']= floors[1]
    if len(l) > 1 :
        result['building_type'] = l[1][1:] # remove first space
    return result


floor_info = rent.apply(get_floor, axis=1, result_type='expand')
floor_info['floor_flat'] = floor_info['floor_flat'].astype('Int64')
floor_info['floors_total'] = floor_info['floors_total'].astype('Int64')
print(floor_info['building_type'].unique())
print("Number of Nan in building_type: ", floor_info['building_type'].isna().sum())  # 2971 – Nan values
print("\nFlat floor vals:", sorted(floor_info['floor_flat'].unique()))    #range -1 to 85
print("\nTotal floors:", sorted(floor_info['floors_total'].unique()))  #range 1 to 116

['Монолитный' 'Монолитно-кирпичный' nan 'Панельный' 'Сталинский'
 'Кирпичный' 'старый фонд' 'Блочный' 'Деревянный' 'Щитовой']
Number of Nan in building_type:  2971

Flat floor vals: [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 59, 61, 65, 70, 71, 75, 85]

Total floors: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 62, 65, 68, 72, 73, 75, 76, 77, 78, 80, 85, 86, 95, 116]


In [243]:
# 'Дополнительно'

def get_equp(row) :
    result = {}
    if type(row['Дополнительно']) != str : return result
    l = row['Дополнительно'].split(',')
    if len(l) == 0 : return result
    for eq in l :
        if eq[0] == ' ' : eq = eq[1:]
        result[eq] = 1
    return result


additional = rent.apply(get_equp, axis=1, result_type='expand')
for c in additional.columns :
    additional[c] = additional[c].astype('Int64')
additional = additional.fillna(0)
print(additional.columns)
additional

Index(['Мебель в комнатах', 'Мебель на кухне', 'Ванна', 'Душевая кабина',
       'Стиральная машина', 'Кондиционер', 'Посудомоечная машина', 'Телевизор',
       'Холодильник', 'Интернет', 'Телефон'],
      dtype='object')


,Мебель в комнатах,Мебель на кухне,Ванна,Душевая кабина,Стиральная машина,Кондиционер,Посудомоечная машина,Телевизор,Холодильник,Интернет,Телефон
0,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
23363,1,1,1,0,1,1,0,0,1,1,0
23364,1,1,1,0,1,0,0,1,1,0,0
23365,0,1,0,1,1,1,0,0,1,1,0
23366,1,1,1,0,1,1,1,0,1,0,0


In [244]:
####################

In [245]:
# 'Балкон'
rent[['balcony', 'loggia']] = rent.Балкон.str.split(",", expand = True)

cou = 0
for i in rent['balcony']:
    new = str(i)
    cou = int(cou)
    if new[0] == 'Л':
        rent.iloc[cou,26] = rent.iloc[cou,25]
        rent.iloc[cou,25] = ''
    cou = cou + 1
    
rent = rent.assign(balcony = lambda x: x['balcony'].str.extract('(\d+)'))
rent = rent.assign(loggia = lambda x: x['loggia'].str.extract('(\d+)'))
rent['balcony'] = rent['balcony'].fillna(0)
rent['loggia'] = rent['loggia'].fillna(0)

IndexError: index 25 is out of bounds for axis 0 with size 23

In [ ]:
# 'Санузел'
rent[['sf1', 'sf2']] = rent.Санузел.str.split(",", expand = True)

cou = 0
for i in rent['sf1']:
    new = str(i)
    cou = int(cou)
    if new[0] == 'Р':
        rent.iloc[cou,30] = rent.iloc[cou,29]
        rent.iloc[cou,29] = ''
    cou = cou + 1
    
rent = rent.assign(sf1 = lambda x: x['sf1'].str.extract('(\d+)'))
rent = rent.assign(sf2 = lambda x: x['sf2'].str.extract('(\d+)'))
rent['sf1'] = df['sf1'].fillna(0)
rent['sf2'] = df['sf2'].fillna(0)
rent = rent.astype({'sf1':'int'})
rent = rent.astype({'sf2':'int'})
rent['bathroom'] = rent['sf1'] + df['sf2'] 
rent.drop(['sf1', 'sf2'], axis=1)

In [ ]:
# 'Лифт'
rent[['passenger_lift', 'service_lift']] = rent.Лифт.str.split(",", expand = True)

cou = 0
for i in rent['passenger_lift']:
    new = str(i)
    cou = int(cou)
    if new[0] == 'Г':
        rent.iloc[cou,28] = rent.iloc[cou,27]
        rent.iloc[cou,28] = ''
    cou = cou + 1
    
rent = rent.assign(passenger_lift = lambda x: x['passenger_lift'].str.extract('(\d+)'))
rent = rent.assign(service_lift = lambda x: x['service_lift'].str.extract('(\d+)'))
rent['passenger_lift'] = rent['passenger_lift'].fillna(0)
rent['service_lift'] = rent['service_lift'].fillna(0)

In [ ]:
#######################

In [ ]:
# 'Площадь, м2'
rent['Площадь, м2'] = rent['Площадь, м2'].str.split(pat='/', n = 1, expand = True)[0].astype('float')
rent.rename(columns = {'Площадь, м2':'square_total'}, inplace = True)
rent[0:2]

In [ ]:
# 'Парковка'
rent['Парковка'] = rent['Парковка'].fillna('нет парковки')
rent.rename(columns = {'Парковка':'parking'}, inplace = True)
rent[0:2]

In [ ]:
# 'Ремонт'
rent['Ремонт'] = rent['Ремонт'].fillna('Без ремонта')
rent.rename(columns = {'Ремонт':'renovation'}, inplace = True)
rent[0:2]

In [ ]:
# 'Окна'
rent['Окна'] = rent['Окна'].fillna('Нет данных')
rent.rename(columns = {'Окна':'windows'}, inplace = True)
rent[0:2]

In [246]:
# 'Можно с детьми/животными'
def get_allowed(row) :
    result = {}
    if type(row['Можно с детьми/животными']) != str : return result
    l = row['Можно с детьми/животными'].split(',')
    if len(l) == 0 : return result
    for al in l :
        if al[0] == ' ' : al = al[1:]
        result[al] = 1
    return result
allowed = rent.apply(get_allowed, axis=1, result_type='expand')
for c in allowed.columns :
    allowed[c] = allowed[c].astype('Int64')
allowed = allowed.fillna(0)
rent[['kids_allowed','pets_allowed']] = allowed
rent.drop('Можно с детьми/животными', axis=1, inplace=True)
rent[0:2]

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,...,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,balcony,loggia,kids_allowed,pets_allowed
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,...,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,NaN,NaN,1,1
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,...,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,NaN,NaN,1,0


In [ ]:
#################

## Extract METRO

In [140]:
# 'Метро'
rent['Станция Метро'] = rent['Метро'].str.split('(').str[0]
rent['Станция Метро']

0        м. Смоленская 
1        м. Смоленская 
2        м. Смоленская 
3        м. Смоленская 
4         м. Арбатская 
              ...      
23363      м. Говорово 
23364      м. Солнцево 
23365      м. Солнцево 
23366      м. Солнцево 
23367      м. Говорово 
Name: Станция Метро, Length: 19737, dtype: object

In [141]:
rent['Расстояние от метро'] = rent['Метро'].str.split('(').str[1]
rent['Расстояние от метро'] = rent['Расстояние от метро'].str[:-1]
rent['Расстояние от метро']

0         9 мин пешком
1         8 мин пешком
2         7 мин пешком
3         3 мин пешком
4         7 мин пешком
             ...      
23363     8 мин пешком
23364     7 мин пешком
23365     6 мин пешком
23366    11 мин пешком
23367    10 мин пешком
Name: Расстояние от метро, Length: 19737, dtype: object

In [142]:
rent['Станция Метро']=rent['Станция Метро'].str[3:-1]

In [143]:
rent.groupby('Станция Метро', as_index=False).agg({'ID  объявления': 'count'})

,Станция Метро,ID объявления
0,Авиамоторная,107
1,Автозаводская,57
2,Академическая,191
3,Александровский сад,3
4,Алексеевская,171
...,...,...
272,Юго-Западная,138
273,Южная,73
274,Ясенево,127
275,Яхромская,1


In [144]:
metroo = metro = pd.read_csv('../MetroAndDistrict/list_of_moscow_metro_stations.csv')
metroo

FileNotFoundError: [Errno 2] No such file or directory: '../MetroAndDistrict/list_of_moscow_metro_stations.csv'

In [ ]:
metro = pd.read_json('datasets/metro.msk.json')
metro

In [ ]:
metro2 = metro['stations'].apply(pd.Series)
metro2

In [ ]:
metro_metro = pd.DataFrame()
x = 0
while x < len(metro2.index):
    pd.concat([metro_metro, metro2.loc[x].apply(pd.Series)], ignore_index=True)
    x += 1
metro_metro    

In [ ]:
metro_metro = pd.concat([metro_metro, metro2.loc[14].apply(pd.Series)], ignore_index=True)
metro_metro

In [ ]:
metro_metro= metro_metro.dropna(how='all')
metro_metro

In [ ]:
metro_metro = metro_metro.drop_duplicates(subset=['name'])
metro_metro


In [ ]:
metro_metro.sort_values(by='name')

In [ ]:
rent.groupby('Станция Метро', as_index=False).agg({'ID  объявления': 'count'})

In [ ]:
metro_metro = metro_metro.rename(columns={'name': 'Станция Метро'})
metro_metro[['Станция Метро', 'admArea']].sort_values('Станция Метро')

In [ ]:
#rent['Станция Метро'] = rent['Станция Метро'].str[2:]
metros = rent.groupby('Станция Метро', as_index=False).agg({'ID  объявления': 'count'})
#metros = metros.rename(columns={'Станция Метро': 'name'})
metros
#metros.merge(metro_metro[['name', 'admArea']].sort_values('name'), how='left')

In [ ]:
pd.merge(metros, metro_metro, on='Станция Метро')#.groupby('admArea', as_index=False).count()

In [ ]:
pd.merge(metros, metro_metro, left_on='Станция Метро', right_on ='Станция Метро', how='left')

In [ ]:
okrugs = pd.merge(rent, metro_metro, left_on='Станция Метро', right_on ='Станция Метро', how='left').groupby('admArea', as_index=False).agg({'ID  объявления': 'count'})
fig, ax = plt.subplots()
ax.pie(okrugs['ID  объявления'], labels=okrugs['admArea'])
ax.get_legend().remove()

In [ ]:
rent.head()

In [ ]:
# 'Количество комнат'

In [ ]:
rent['Число комнат'] = rent['Количество комнат'].str.split(',').str[0]
rent

In [ ]:
rent['Число комнат'] = pd.to_numeric(rent['Число комнат'])

In [ ]:
rent['Тип комнат'] = rent['Количество комнат'].str.split(',').str[1]
rent

In [ ]:
rent = pd.merge(rent, metro_metro[['Станция Метро', 'admArea']], on='Станция Метро', how='left')
rent


In [ ]:
rent = rent.rename(columns={'admArea': 'Округ'})
rent

KeyError: 'Площадь, м2'

In [ ]:
# 'Цена'


In [ ]:
# 'Описание'


In [ ]:
# 'Площадь комнат, м2'


In [ ]:
# 'Название ЖК'


In [ ]:
# 'Серия дома'


In [ ]:
# 'Высота потолков, м'


In [ ]:
# 'Мусоропровод'
